In [22]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import PIL
import pytesseract
import spacy
import os

In [2]:
img_cv = cv2.imread('E:/FILE of Trong/NLP Project/ner/data/052.jpeg')
cv2.imshow('BusinessCard', img_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
img_pl = PIL.Image.open('E:/FILE of Trong/NLP Project/ner/data/002.jpeg')

In [4]:
type(img_pl)

PIL.JpegImagePlugin.JpegImageFile

In [5]:
# img_pl

In [6]:
# img_cv = cv2.imread('E:/FILE of Trong/NLP Project/ner/data_vn/test_vn.jpg')
# cv2.imshow('BusinessCard', img_cv)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Text from Image

In [7]:
text_cv = pytesseract.image_to_string(img_cv)
print(text_cv)

HONDA

DONNIE HANSEN
MOTOCROSS TEAM RIDER
TEAM HONDA

AMERICAN HONDA MOTOR CO., INC.
100 WEST ALONDRA BOULEVARD e GARDENA, CA 90247
(213) 321-8680



# Image to Data

In [18]:
data = pytesseract.image_to_data(img_cv)

In [19]:
data

'level\tpage_num\tblock_num\tpar_num\tline_num\tword_num\tleft\ttop\twidth\theight\tconf\ttext\n1\t1\t0\t0\t0\t0\t0\t0\t720\t401\t-1\t\n2\t1\t1\t0\t0\t0\t36\t29\t207\t25\t-1\t\n3\t1\t1\t1\t0\t0\t36\t29\t207\t25\t-1\t\n4\t1\t1\t1\t1\t0\t36\t29\t207\t25\t-1\t\n5\t1\t1\t1\t1\t1\t36\t29\t207\t25\t95\tHONDA\n2\t1\t2\t0\t0\t0\t237\t166\t246\t63\t-1\t\n3\t1\t2\t1\t0\t0\t237\t166\t246\t63\t-1\t\n4\t1\t2\t1\t1\t0\t237\t166\t246\t16\t-1\t\n5\t1\t2\t1\t1\t1\t237\t166\t112\t16\t96\tDONNIE\n5\t1\t2\t1\t1\t2\t364\t166\t119\t16\t96\tHANSEN\n4\t1\t2\t1\t2\t0\t245\t194\t231\t12\t-1\t\n5\t1\t2\t1\t2\t1\t245\t195\t108\t11\t96\tMOTOCROSS\n5\t1\t2\t1\t2\t2\t362\t194\t48\t11\t96\tTEAM\n5\t1\t2\t1\t2\t3\t422\t194\t54\t11\t96\tRIDER\n4\t1\t2\t1\t3\t0\t299\t218\t123\t11\t-1\t\n5\t1\t2\t1\t3\t1\t299\t218\t49\t11\t94\tTEAM\n5\t1\t2\t1\t3\t2\t359\t218\t63\t11\t96\tHONDA\n2\t1\t3\t0\t0\t0\t35\t304\t649\t67\t-1\t\n3\t1\t3\t1\t0\t0\t35\t304\t649\t67\t-1\t\n4\t1\t3\t1\t1\t0\t35\t304\t648\t22\t-1\t\n5\t1\t3\t1\t1\t1\t

In [20]:
data_list = list(map(lambda x: x.split('\t'), data.split('\n')))

In [23]:
df = pd.DataFrame(data_list[1:], columns = data_list[0])

In [25]:
df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,720,401,-1,
1,2,1,1,0,0,0,36,29,207,25,-1,
2,3,1,1,1,0,0,36,29,207,25,-1,
3,4,1,1,1,1,0,36,29,207,25,-1,
4,5,1,1,1,1,1,36,29,207,25,95,HONDA


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   level      38 non-null     object
 1   page_num   37 non-null     object
 2   block_num  37 non-null     object
 3   par_num    37 non-null     object
 4   line_num   37 non-null     object
 5   word_num   37 non-null     object
 6   left       37 non-null     object
 7   top        37 non-null     object
 8   width      37 non-null     object
 9   height     37 non-null     object
 10  conf       37 non-null     object
 11  text       37 non-null     object
dtypes: object(12)
memory usage: 3.7+ KB


In [27]:
df.dropna(inplace = True)
col_int = ['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf']
df[col_int] = df[col_int].astype(int)

In [28]:
df.dtypes

level         int32
page_num      int32
block_num     int32
par_num       int32
line_num      int32
word_num      int32
left          int32
top           int32
width         int32
height        int32
conf          int32
text         object
dtype: object

In [38]:
image = img_cv.copy()
level = 'word'
for l, x, y, w, h, c, txt in df[['level', 'left', 'top', 'width', 'height','conf', 'text']].values:
    # print(l, x, y, w, h, c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0,0,0), 2)
        else:
            continue
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image, (x, y), (x+w, y+h), (255,0,0), 2)
        else:
            continue
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0), 2)
        else:
            continue
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0), 2)
        else:
            continue
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0),2)
            cv2.putText(image, txt, (x,y), cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
cv2.imshow("Bounding Box", image)
cv2.waitKey()
cv2.destroyAllWindows()